# Filipe Borba e Martim José

In [1]:
import sys
sys.path.append('..')

from random import choice
import pandas as pd
import scipy
import numpy as np
from itertools import permutations
import networkx as nx
import seaborn as sns
import matplotlib.pyplot as plt

import socnet as sn

In [2]:
sn.node_size = 10
sn.node_color = (255, 255, 255)

sn.edge_width = 1
sn.edge_color = (192, 192, 192)

sn.node_label_position = 'top center'

In [3]:
g = sn.load_graph('one-mode.gml', has_pos=False)

In [4]:
w = sn.load_graph('two-mode.gml', has_pos=False)

In [5]:
def set_indegree(g):
    for n in g.nodes:
        g.nodes[n]['indegree'] = g.in_degree(n)
        
def set_out_degree(g):
    for n in g.nodes:
        g.nodes[n]['outdegree'] = g.out_degree(n)

In [6]:
def outdegree_ponderado(g):
    for n in g.nodes:
        node = g.nodes[n]
        if node['indegree'] == 0: #É uma pessoa
            soma = 0
            for j in range(len(list(g.successors(n)))):
                soma += g.nodes[list(g.successors(n))[j]]["indegree"]
            node["hub"] = soma
        else:
            node["hub"] = 0
    return g

def indegree_ponderado(g):
    for n in g.nodes:
        node = g.nodes[n]
        if node['outdegree'] == 0: #É um genero
            soma = 0
            for j in range(len(list(g.predecessors(n)))):
                soma += g.nodes[list(g.predecessors(n))[j]]["hub"]
            node["auth"] = soma
        else:
            node["auth"] = 0
    return g

def indegree_ponderado_normalizado(g):
    soma_total = 0
    for n in g.nodes:
        if g.nodes[n]["indegree"] == 0:
            soma_total += g.nodes[n]["hub"]
        
    for n in g.nodes:
        if g.nodes[n]["indegree"] == 0:
            g.nodes[n]["hub"] = g.nodes[n]["hub"]/soma_total
    return g
        
def outdegree_ponderado_normalizado(g):
    soma_total = 0
    for n in g.nodes:
        if g.nodes[n]['outdegree'] == 0:
            soma_total += g.nodes[n]["auth"]
        
    for n in g.nodes:
        if g.nodes[n]['outdegree'] == 0:
            g.nodes[n]["auth"] = g.nodes[n]["auth"]/soma_total
    return g

def euclidian_distance(g1, g2):
    hub = sum([((g1.nodes[n]["hub"] - g2.nodes[n]["hub"])**2) for n in g.nodes])**0.5
    print(hub)
    auth = sum([(g1.nodes[i]["auth"] - g2.nodes[i]["auth"])** 2 for i in range(len(g1.nodes))]) ** 0.5
    return hub
    
def set_hub_authority(g):
    set_indegree(g)
    set_out_degree(g)
    g = outdegree_ponderado(g)
    g = indegree_ponderado(g)
    g = indegree_ponderado_normalizado(g)
    g = outdegree_ponderado_normalizado(g)
    return g

def converge(g):
    g = set_hub_authority(g)
    delta = 1
    epsilon = 10**-7
    while delta > epsilon:
        g1 = g.copy()
        g = set_hub_authority(g)
        delta = euclidian_distance(g1, g)
    return g

In [7]:
converge(w)
pr2 = nx.pagerank(w, alpha=0.9)

0.0


In [8]:
twomode = pd.DataFrame({
    'nome': [w.nodes[n]['label'] for n in w.nodes],
    'outdegree': [w.nodes[n]['outdegree'] for n in w.nodes],
    'indegree': [w.nodes[n]['indegree'] for n in w.nodes],
    'hub': [w.nodes[n]['hub'] for n in w.nodes],
    'auth': [w.nodes[n]['auth'] for n in w.nodes],
    'pageranks': [pr2[n] for n in pr2]
})

In [9]:
twomode[twomode.indegree != 0].iloc[:, [0,2,4]].sort_values(by="auth", ascending=False)

,nome,indegree,auth
24,Eletronica,19,0.128628
32,Rock classico,11,0.079566
25,Pop Rock,11,0.078085
5,Pop,11,0.075025
31,Pagode,10,0.072655
4,Rap Nacional,10,0.068806
17,Indie Rock,8,0.063672
26,Disco,8,0.063376
19,Funk,9,0.058243
22,Sertanejo,8,0.057947


In [10]:
twomode[twomode.outdegree != 0].iloc[:, [0,1,3]].sort_values(by="hub", ascending=False)

,nome,outdegree,hub
12,Forba,15,0.091625
34,Jean,10,0.068003
23,Sabrina,10,0.067287
21,Toyama,8,0.059413
20,Rachel,7,0.055834
16,Vitoria,7,0.052971
13,Hugo,7,0.051539
1,Isa,7,0.050823
33,Joao,6,0.047960
39,Graicer,6,0.046528


In [11]:
twomode[twomode.indegree != 0].iloc[:, [0,5]].sort_values(by="pageranks", ascending=False)

,nome,pageranks
24,Eletronica,0.065738
5,Pop,0.049658
25,Pop Rock,0.046922
37,J-Pop,0.042933
32,Rock classico,0.042192
4,Rap Nacional,0.041450
31,Pagode,0.038662
19,Funk,0.038644
22,Sertanejo,0.034549
15,Metal,0.034037


In [12]:
hub_list = []
for i in w.nodes:
    value = w.nodes[i]["hub"]
    hub_list.append(value)

auth_list = []
for i in w.nodes:
    value = w.nodes[i]["auth"]
    auth_list.append(value)
    
pr_list = []
for i in pr2:
    pr_list.append(i)

In [13]:
from scipy.stats import linregress

def get_indegree_list(g):
    indegree_list = []
    for e in g.nodes:
        value = g.nodes[e]["indegree"]
        indegree_list.append(value)
    return indegree_list

def get_outdegree_list(g):
    outdegree_list = []
    for e in g.nodes:
        value = g.nodes[e]["outdegree"]
        outdegree_list.append(value)
    return outdegree_list

indegree_list = get_indegree_list(w)
outdegree_list = get_outdegree_list(w)

l = linregress(outdegree_list, hub_list)
print(l,"\n")

l = linregress(indegree_list, auth_list)
print(l,"\n")

LinregressResult(slope=0.006816648195899725, intercept=0.0013121475192484564, rvalue=0.9875404579798647, pvalue=3.9527949934280903e-32, stderr=0.0001762110803946884) 

LinregressResult(slope=0.007031460889991428, intercept=0.0005656734072797889, rvalue=0.9966908934679561, pvalue=4.908726682803212e-43, stderr=9.302594061392511e-05) 



# One Mode

In [14]:
hubs, auth = nx.hits(g)
set_indegree(g)
set_out_degree(g)
pr = nx.pagerank(g, alpha=0.9)

In [15]:
pd.DataFrame({
    'nome': [g.nodes[n]['label'] for n in g.nodes],
    'indegree': [g.nodes[n]['indegree'] for n in g.nodes],
    'outdegree': [g.nodes[n]['outdegree'] for n in g.nodes],
    'hub': [hubs[n] for n in hubs],
    'auth': [auth[n] for n in auth],
    'pageranks': [pr[n] for n in pr]
}).sort_values(by=["pageranks", "auth"], ascending=False)

,nome,indegree,outdegree,hub,auth,pageranks
10,Hugo,12,3,0.038989,0.135274,0.183554
18,Graicer,10,2,0.034679,0.125647,0.105987
15,Forba,7,7,0.102443,0.100104,0.098895
7,Fred,7,4,0.056536,0.096399,0.095981
22,Isa,3,2,0.035588,0.040415,0.076960
0,Rachel,5,4,0.047178,0.078124,0.068358
8,Toyama,7,2,0.036157,0.109203,0.063825
12,Elisa,2,1,0.002895,0.018846,0.058707
23,Chen,6,3,0.027634,0.036790,0.055260
9,Vitoria,5,4,0.013276,0.039265,0.027585
